In [ ]:
%pip install azure-cosmos
%pip install faker

In [11]:
from azure.cosmos import CosmosClient, PartitionKey
from configparser import ConfigParser
from faker import Faker
from collections import OrderedDict

import os
import json
import uuid
import datetime


parser = ConfigParser()
parser.read('../NotebookConfig.cfg')

cosmosAccountURI = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_URI')
cosmosAccountKey = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_KEY')

databaseName = 'Learn'
containerName = 'AISearch'
partitionKeypath = '/PartitionKey'

Faker.seed(0)
fake = Faker(['en-US'])

print(cosmosAccountURI)

https://cosmicgbbcdb-sql.documents.azure.com:443/


In [17]:
client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName)

pkPath = PartitionKey(path=partitionKeypath)
ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath, offer_throughput=400, default_ttl=-1) 

# Default TTL is set to -1, which means the documents will never expire unless you set a value in seconds

In [18]:
maxrange = 100

for j in range(maxrange):
    docId = str(uuid.uuid4())

    doc = {
        'id': docId
        , 'PartitionKey': docId
        , 'Status': 'Active'
        , 'Name': fake.name()
        , 'Title': fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None)
        , 'Description': fake.sentence(nb_words=100, variable_nb_words=True, ext_word_list=None)
        , 'Class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
        , 'UpdateTS': datetime.datetime.now().timestamp()
    }
    
    ctr.create_item(doc)

print('Completed writing ' + str(maxrange) + ' documents to CosmosDB')

Completed writing 100 documents to CosmosDB


In [22]:
queryResult = ctr.query_items( query='SELECT TOP 5 * FROM c ORDER BY c._ts ASC', enable_cross_partition_query=True, max_item_count=5)
for doc in queryResult: 
    doc['Status'] = 'Deleted'
    doc['UpdateTS']: datetime.datetime.now().timestamp()
    doc['ttl'] = 900 # 15 minutes for test, will add a lot more for production to ensure indexer can catchup and update index before TTL expiration

    ctr.upsert_item(doc)
    print(doc)    

{'id': '6be37da0-5b68-4ae5-b873-71e860abd252', 'PartitionKey': '6be37da0-5b68-4ae5-b873-71e860abd252', 'Status': 'Deleted', 'Name': 'George Berg', 'Title': 'Well health table this skin reflect.', 'Description': 'Much more federal suffer rich seem plant prepare evidence certain for everybody society among less someone whatever goal nation somebody sometimes skin relationship moment air director base piece mother administration from health claim moment seat treatment star large two leader foot your art return college country peace chair gas participant successful and western in book buy information life near force group all lead hard poor maybe structure public include return bed style board reflect wear similar Congress economic option feel fill about article lead air carry writer hard indeed example part area fire fight choice factor close national generation young seat dinner majority last rich education I note really available public begin wear surface cup indeed certainly everyone r

### Clean up code

In [16]:
# Assume objects are instantiated
db.delete_container(containerName)